In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [140]:
import pandas as pd
from app.data_processing.utils import *
from app.data_processing.equivalences import equivalences

In [10]:
testing_recipes = [74146, 73727, 74020 , 73919, 73818, 73799,73756, 73360, 73314, 73269, 72692, 72311, 72126,66092, 65997, 71629]

In [56]:
def flatten(list_of_lists):
    return [item for list in list_of_lists for item in list]

In [20]:
df = pd.read_csv('data/output.csv', sep='|')
recipes = df[["Id","Ingredientes"]].copy()
recipes = recipes[recipes['Id'].isin(testing_recipes)]

In [338]:
ingredients = pd.read_csv('data/bedca.csv',index_col=0)
ingredients_names = ingredients["nombre"].tolist()
ingredients['Id'] = ingredients.index

In [116]:
all_ingredients = ingredients_to_plural(ingredients_names)  # ingredientes originales + plurales
bedca_ingredients = [x.lower() for x in all_ingredients]
recipes.reset_index(drop=True, inplace=True)

In [232]:
ing = recipes['Ingredientes'].iloc[0]

In [233]:
parsed_ingredients, rate, total, extracted = parse_ingredient_string(ing, bedca_ingredients)

In [242]:
def remove_list(row):
    if isinstance(row,list):
        row = row[0]
    return row

In [259]:
test = pd.DataFrame(parsed_ingredients)
test = test.drop([1,3,4,10])
test['Cantidad'] = test['Cantidad'].apply(remove_list)
test['Cantidad'] = test['Cantidad'].astype(float)
test['Ingrediente'] = test['Ingrediente'].apply(remove_list)
test['Unidad'] = test['Unidad'].apply(remove_list)

test['Indice'] = [243,419,22,507,907,483,404]

test


,Ingrediente,Cantidad,Unidad,Indice
0,pollo,1.0,pechuga,243
2,mantequilla,150.0,gramos,419
5,leche,2.0,vasos,22
6,pan rallado,1.0,tarro,507
7,sal,1.0,pellizco,907
8,harina,3.0,cucharadas soperas,483
9,aceite de girasol,2.0,vasos,404


In [260]:
def get_grams(row):
    row = equivalences[row] 
    return row

In [261]:
test['Grams'] = test['Unidad'].apply(get_grams)
test['Total_Grams'] = test['Grams'] * test['Cantidad']

In [263]:
test

,Ingrediente,Cantidad,Unidad,Indice,Grams,Total_Grams
0,pollo,1.0,pechuga,243,400,400.0
2,mantequilla,150.0,gramos,419,1,150.0
5,leche,2.0,vasos,22,250,500.0
6,pan rallado,1.0,tarro,507,250,250.0
7,sal,1.0,pellizco,907,1,1.0
8,harina,3.0,cucharadas soperas,483,17,51.0
9,aceite de girasol,2.0,vasos,404,250,500.0


In [340]:
#macro_columns = ['energía','grasa','proteina','fibra','carbohidratos']
categorical_columns = ['nombre','categoria']

def extract_recipe_value(ingredients):
    '''
    :return result_df: df which contains the nutritional values of the total serving 
    '''
    nutri_values = get_df_by_ids(ingredients['Indice']).drop(columns = categorical_columns)
    nutri_columns = nutri_values.columns
    join = test.join(nutri_values, on = 'Indice')
    join[nutri_columns] = join[nutri_columns].multiply(join['Total_Grams']/100, axis='index')
    result_df = join[nutri_columns].sum().round(6)
    result_df['Gramos'] = join['Total_Grams'].sum()

    return result_df
    
def get_df_by_ids(ids):
    ids = [i for i in ids if i != None]
    return ingredients[ingredients['Id'].isin(ids)]

In [341]:
extract_recipe_value(test)

alcohol                                                                           0.000000
energía                                                                        6746.220000
grasa                                                                           611.062106
proteina                                                                        139.690000
agua                                                                            901.870000
fibra                                                                            13.340000
carbohidratos                                                                   184.176000
ácido graso 22:6 n-3 (ácido docosahexaenóico)                                     0.000000
ácidos grasos, monoinsaturados totales                                          158.318000
ácidos grasos, poliinsaturados totales                                          323.699000
ácidos grasos saturados totales                                                 118.539700